In [144]:
%reset 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

filepath = "../data/digester_data.csv"
dataset = pd.read_csv(filepath, index_col=0, parse_dates=True)
dataset.describe()

,Wash Filtrate to Digester - Total Flow,Wash Filtrate Vertical Nozzle,White Liquor Flow to ITC,H-factor,H-factor Co-Current,H-factor Target,H-factor Counter-Current,Volumetric Chip Flow,Chip Mass Flow from Piles,Spruce Saw Mill Chips Share,...,Hot Spent Digester Liquor Specific Flow to Impregnation Vessel,Digester Production,Digester Long Level,Digester Chip Level Control,Digester Dilution Factor,Digester Short Level,Blow Kappa,Blow Temperature,Blow Production,Weak Black Liquor to Evaporation Plant Residual Alkali
count,5968.000000,5968.000000,5968.000000,5968.000000,5968.000000,5968.000000,5968.000000,5968.000000,5968.000000,5968.000000,...,5968.000000,5.968000e+03,5968.000000,5.968000e+03,5968.000000,5968.000000,5968.000000,5968.000000,5968.000000,5968.000000
mean,152.684728,136.377225,0.805236,570.977032,301.579641,570.424631,269.397390,648.486237,36.875105,8.321328,...,2.699426,1.131095e+03,88.548976,1.821106e+01,2.180605,48.323078,27.805850,87.668116,1192.618513,10.258447
std,16.048149,13.919758,0.065796,44.603387,23.764770,42.663694,23.083611,151.769241,4.564046,13.778885,...,0.024933,9.648935e+01,0.389135,1.198734e+01,0.077165,0.930924,1.542807,1.430514,110.261327,0.886624
min,74.308880,67.062810,0.247412,481.332200,238.832000,510.000000,218.241900,0.287224,0.000000,0.000000,...,1.557932,2.920000e-24,85.750780,8.270000e-44,0.562290,38.517540,22.170250,78.215330,617.716000,8.920000
25%,150.227000,134.500125,0.812160,533.705400,281.981575,530.000000,251.153225,657.000750,34.966920,0.000000,...,2.700000,1.159972e+03,88.269143,9.788168e+00,2.112270,47.825635,26.894120,86.934745,1173.334500,9.410000
50%,155.692000,139.346850,0.821603,559.434800,293.838550,550.000000,265.667550,690.155350,37.225650,0.000000,...,2.700000,1.164592e+03,88.548210,1.743239e+01,2.192728,48.213135,27.760630,87.685415,1214.454500,10.190000
75%,161.757025,143.924075,0.833718,609.275650,321.322500,600.000000,287.128975,711.669425,40.014853,14.973385,...,2.700000,1.165070e+03,88.808513,2.506831e+01,2.244194,48.771113,28.699717,88.671003,1251.014500,11.080000
max,190.484000,170.756900,0.912871,732.269400,376.860700,670.000000,364.774900,865.293400,48.994150,50.678790,...,2.800951,1.195267e+03,93.199450,1.015897e+02,2.727024,66.054000,33.682950,90.681490,1500.449000,11.370000


In [145]:
y_data = dataset['Blow Kappa'].to_numpy()
X_data = dataset.to_numpy()
print(y_data.shape)
print(X_data.shape)

(5968,)
(5968, 37)


In [146]:
# Base Line

y_data_B = np.full(
    shape=len(y_data),
    fill_value=np.mean(y_data),
    dtype=np.float64
)

In [147]:
#Split des données en train et test

y_train_B = y_data_B[:int(len(y_data_B)*0.8)]
y_test_B = y_data_B[int(len(y_data_B)*0.8):]

y_train = y_data[:int(len(y_data)*0.8)]
y_test = y_data[int(len(y_data)*0.8):]

X_train = X_data[:int(len(X_data)*0.8)]
X_test = X_data[int(len(X_data)*0.8):]

print(y_train.shape, y_test.shape)
print(X_train.shape, X_test.shape)

(4774,) (1194,)
(4774, 37) (1194, 37)


In [148]:

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train)

In [138]:
from sklearn.svm import SVR
# import GridSearchCV
from sklearn.model_selection import GridSearchCV

# param = {
#     'kernel': ['rbf'],
#     'epsilon': [0.01, 0.05, 0.1, 0.5, 1],
#     'gamma': [0.01, 0.1, 1],
#     'C': [1, 10, 100]
# }

# grid = GridSearchCV(SVR(), param, cv=5)

In [139]:
# grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 10, 100],
                         'epsilon': [0.01, 0.05, 0.1, 0.5, 1],
                         'gamma': [0.01, 0.1, 1], 'kernel': ['rbf']})

In [140]:
# print(grid.best_params_)

{'C': 100, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}


In [151]:
from sklearn.svm import SVR

regressor = SVR(kernel='rbf', epsilon=0.01, gamma=0.01,C=100)

regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

print(y_pred)

[27.70333107 27.70333107 27.70333107 ... 27.70333107 27.70333107
 27.70333107]


In [152]:
from sklearn import metrics

print(metrics.mean_absolute_error(y_test, y_pred))
print("Base Line : ",metrics.mean_absolute_error(y_train, y_train_B))


1.2873748408615433
Base Line :  1.1396043875364883


In [153]:
#cross-validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(regressor, X_train, y_train, cv=5)

print(scores)

[0.97808589 0.98407713 0.9322739  0.99942284 0.98149782]
